In [13]:
# Here I am using gensim for learning the word embeddings from the EUADR corpus
# This is just a rough start to learn 
import gensim
import logging
import os
import pandas as pd

In [14]:
# start the log
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

In [15]:
# download NLTK basic stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_my_text(full_text):
    
    
    stopset = set(stopwords.words('english')) # | set(string.punctuation)
    tokens = nltk.word_tokenize(full_text)
    cleanup = [token for token in tokens if token not in stopset and len(token)>2]
    
    return cleanup


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/synoptica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# get the current working directory and file
data_path = '/home/stirunag/OTAR_Dataset/OTAR_sentences.csv'

class MyCorpus(object):
    def __iter__(self):
        for line in open(data_path, encoding='ISO-8859-1'):
            # assume there's one document per line, tokens separated by whitespace
            yield clean_my_text(line)
#             yield line.lower().split()
            
            
# iterate one line at a time and learn embeddings 
sentences = MyCorpus() # a memory-friendly iterator
# model = gensim.models.Word2Vec(sentences)  

model = gensim.models.Word2Vec(
    sentences,
    size=200,
    window=5,
    min_count=2,
    workers=4, iter=10)


logging.info("Done training data files")

2019-08-12 20:21:17,120 : INFO : collecting all words and their counts
2019-08-12 20:21:17,122 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-12 20:21:20,641 : INFO : PROGRESS: at sentence #10000, processed 170845 words, keeping 19356 word types
2019-08-12 20:21:24,030 : INFO : PROGRESS: at sentence #20000, processed 339054 words, keeping 30762 word types
2019-08-12 20:21:27,472 : INFO : PROGRESS: at sentence #30000, processed 506661 words, keeping 40650 word types
2019-08-12 20:21:30,912 : INFO : PROGRESS: at sentence #40000, processed 672032 words, keeping 50097 word types
2019-08-12 20:21:34,365 : INFO : PROGRESS: at sentence #50000, processed 834870 words, keeping 58363 word types
2019-08-12 20:21:37,756 : INFO : PROGRESS: at sentence #60000, processed 1002902 words, keeping 66765 word types
2019-08-12 20:21:41,178 : INFO : PROGRESS: at sentence #70000, processed 1175301 words, keeping 74771 word types
2019-08-12 20:21:44,565 : INFO : PROGRESS: a

2019-08-12 20:25:34,257 : INFO : PROGRESS: at sentence #710000, processed 13246147 words, keeping 629204 word types
2019-08-12 20:25:38,037 : INFO : PROGRESS: at sentence #720000, processed 13458288 words, keeping 638470 word types
2019-08-12 20:25:41,800 : INFO : PROGRESS: at sentence #730000, processed 13667846 words, keeping 649040 word types
2019-08-12 20:25:45,680 : INFO : PROGRESS: at sentence #740000, processed 13894371 words, keeping 658406 word types
2019-08-12 20:25:49,612 : INFO : PROGRESS: at sentence #750000, processed 14121254 words, keeping 668679 word types
2019-08-12 20:25:55,503 : INFO : PROGRESS: at sentence #760000, processed 14545179 words, keeping 679359 word types
2019-08-12 20:25:59,984 : INFO : PROGRESS: at sentence #770000, processed 14827121 words, keeping 690887 word types
2019-08-12 20:26:04,116 : INFO : PROGRESS: at sentence #780000, processed 15066747 words, keeping 702938 word types
2019-08-12 20:26:08,090 : INFO : PROGRESS: at sentence #790000, processe

2019-08-12 20:30:37,107 : INFO : PROGRESS: at sentence #1410000, processed 30691580 words, keeping 1374915 word types
2019-08-12 20:30:41,477 : INFO : PROGRESS: at sentence #1420000, processed 30935525 words, keeping 1384551 word types
2019-08-12 20:30:45,694 : INFO : PROGRESS: at sentence #1430000, processed 31175862 words, keeping 1394634 word types
2019-08-12 20:30:49,975 : INFO : PROGRESS: at sentence #1440000, processed 31420207 words, keeping 1404643 word types
2019-08-12 20:30:54,256 : INFO : PROGRESS: at sentence #1450000, processed 31682249 words, keeping 1413429 word types
2019-08-12 20:30:58,329 : INFO : PROGRESS: at sentence #1460000, processed 31911865 words, keeping 1422612 word types
2019-08-12 20:31:02,824 : INFO : PROGRESS: at sentence #1470000, processed 32188993 words, keeping 1433632 word types
2019-08-12 20:31:06,929 : INFO : PROGRESS: at sentence #1480000, processed 32427528 words, keeping 1443400 word types
2019-08-12 20:31:11,195 : INFO : PROGRESS: at sentence #

2019-08-12 20:35:59,833 : INFO : PROGRESS: at sentence #2110000, processed 51512590 words, keeping 2043706 word types
2019-08-12 20:36:07,474 : INFO : PROGRESS: at sentence #2120000, processed 51970661 words, keeping 2053165 word types
2019-08-12 20:36:14,784 : INFO : PROGRESS: at sentence #2130000, processed 52367134 words, keeping 2061262 word types
2019-08-12 20:36:19,028 : INFO : PROGRESS: at sentence #2140000, processed 52615955 words, keeping 2070634 word types
2019-08-12 20:36:23,286 : INFO : PROGRESS: at sentence #2150000, processed 52864506 words, keeping 2079900 word types
2019-08-12 20:36:27,476 : INFO : PROGRESS: at sentence #2160000, processed 53107623 words, keeping 2089533 word types
2019-08-12 20:36:31,548 : INFO : PROGRESS: at sentence #2170000, processed 53351207 words, keeping 2097912 word types
2019-08-12 20:36:35,623 : INFO : PROGRESS: at sentence #2180000, processed 53583193 words, keeping 2106172 word types
2019-08-12 20:36:39,676 : INFO : PROGRESS: at sentence #

2019-08-12 20:41:00,982 : INFO : PROGRESS: at sentence #2810000, processed 69024888 words, keeping 2592652 word types
2019-08-12 20:41:05,304 : INFO : PROGRESS: at sentence #2820000, processed 69252572 words, keeping 2600175 word types
2019-08-12 20:41:09,530 : INFO : PROGRESS: at sentence #2830000, processed 69506727 words, keeping 2606581 word types
2019-08-12 20:41:14,033 : INFO : PROGRESS: at sentence #2840000, processed 69738673 words, keeping 2613946 word types
2019-08-12 20:41:18,183 : INFO : PROGRESS: at sentence #2850000, processed 69982133 words, keeping 2621655 word types
2019-08-12 20:41:22,213 : INFO : PROGRESS: at sentence #2860000, processed 70196733 words, keeping 2627423 word types
2019-08-12 20:41:26,497 : INFO : PROGRESS: at sentence #2870000, processed 70435370 words, keeping 2634857 word types
2019-08-12 20:41:31,103 : INFO : PROGRESS: at sentence #2880000, processed 70746597 words, keeping 2641409 word types
2019-08-12 20:41:35,386 : INFO : PROGRESS: at sentence #

...

2019-08-12 21:07:34,234 : INFO : EPOCH 1 - PROGRESS: at 3.83% examples, 44814 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:35,248 : INFO : EPOCH 1 - PROGRESS: at 3.88% examples, 44841 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:36,272 : INFO : EPOCH 1 - PROGRESS: at 3.92% examples, 44861 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:37,324 : INFO : EPOCH 1 - PROGRESS: at 3.97% examples, 44864 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:38,350 : INFO : EPOCH 1 - PROGRESS: at 4.02% examples, 44882 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:39,551 : INFO : EPOCH 1 - PROGRESS: at 4.07% examples, 44919 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:40,556 : INFO : EPOCH 1 - PROGRESS: at 4.12% examples, 44944 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:41,773 : INFO : EPOCH 1 - PROGRESS: at 4.17% examples, 44969 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:07:42,781 : INFO : EPOCH 1 - PROGRESS: at 4.22% examples, 44993 words/s, in_qsize 0, out_qsize 0
2

2019-08-12 21:08:54,918 : INFO : EPOCH 1 - PROGRESS: at 7.38% examples, 45919 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:08:56,101 : INFO : EPOCH 1 - PROGRESS: at 7.43% examples, 45933 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:08:57,112 : INFO : EPOCH 1 - PROGRESS: at 7.47% examples, 45939 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:08:58,119 : INFO : EPOCH 1 - PROGRESS: at 7.52% examples, 45945 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:08:59,304 : INFO : EPOCH 1 - PROGRESS: at 7.57% examples, 45960 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:09:00,458 : INFO : EPOCH 1 - PROGRESS: at 7.62% examples, 45984 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:09:01,486 : INFO : EPOCH 1 - PROGRESS: at 7.67% examples, 45985 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:09:02,511 : INFO : EPOCH 1 - PROGRESS: at 7.71% examples, 45987 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:09:03,521 : INFO : EPOCH 1 - PROGRESS: at 7.75% examples, 45993 words/s, in_qsize 0, out_qsize 0
2

2019-08-12 21:10:18,291 : INFO : EPOCH 1 - PROGRESS: at 10.83% examples, 46717 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:19,430 : INFO : EPOCH 1 - PROGRESS: at 10.88% examples, 46731 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:20,548 : INFO : EPOCH 1 - PROGRESS: at 10.93% examples, 46750 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:21,681 : INFO : EPOCH 1 - PROGRESS: at 10.97% examples, 46765 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:22,784 : INFO : EPOCH 1 - PROGRESS: at 11.02% examples, 46785 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:23,838 : INFO : EPOCH 1 - PROGRESS: at 11.06% examples, 46815 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:24,847 : INFO : EPOCH 1 - PROGRESS: at 11.10% examples, 46814 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:26,019 : INFO : EPOCH 1 - PROGRESS: at 11.15% examples, 46821 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:10:27,150 : INFO : EPOCH 1 - PROGRESS: at 11.19% examples, 46837 words/s, in_qsize 0, out_

2019-08-12 21:11:40,145 : INFO : EPOCH 1 - PROGRESS: at 13.82% examples, 47949 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:11:41,147 : INFO : EPOCH 1 - PROGRESS: at 13.86% examples, 47946 words/s, in_qsize 0, out_qsize 0
2019-08-12 21:11:42,223 : INFO : EPOCH 1 - PROGRESS: at 13.90% examples, 47963 words/s, in_qsize 0, out_qsize 0


KeyboardInterrupt: 

In [26]:
w1 = "gut"

print("Most similar to {0}".format(w1), model.wv.most_similar(positive=w1))

2019-08-12 21:11:48,690 : INFO : precomputing L2-norms of word weight vectors


Most similar to gut [('sacs', 0.9412724375724792), ('vitro', 0.8046704530715942), ('respect', 0.7879438996315002), ('Southern', 0.7832915782928467), ('showed', 0.7751600742340088), ('vivo', 0.7353862524032593), ('detectable', 0.7232041954994202), ('analysis', 0.7121514081954956), ('RT-PCR', 0.707546591758728), ('blotting', 0.7064051032066345)]


In [27]:
w1 = "BPAG2"

print("Most similar to {0}".format(w1), model.wv.most_similar(positive=w1))

Most similar to BPAG2 [('K18', 0.9340334534645081), ('besides', 0.9031471610069275), ('epidermal', 0.8364163041114807), ('patterns', 0.8107117414474487), ('aim', 0.7890700101852417), ('PCR-templates', 0.7804863452911377), ('A=alert', 0.769689679145813), ('MMP-9', 0.7669159770011902), ('MT1-MMP', 0.7651901245117188), ('evaluates', 0.7509801387786865)]


In [28]:
model = gensim.models.Word2Vec(
    sentences,
    size=200,
    window=5,
    min_count=2,
    workers=14, iter=10)


logging.info("Done training data files")

2019-08-12 21:12:19,742 : INFO : collecting all words and their counts
2019-08-12 21:12:19,744 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-12 21:12:23,252 : INFO : PROGRESS: at sentence #10000, processed 170845 words, keeping 19356 word types
2019-08-12 21:12:26,666 : INFO : PROGRESS: at sentence #20000, processed 339054 words, keeping 30762 word types
2019-08-12 21:12:30,145 : INFO : PROGRESS: at sentence #30000, processed 506661 words, keeping 40650 word types
2019-08-12 21:12:33,646 : INFO : PROGRESS: at sentence #40000, processed 672032 words, keeping 50097 word types
2019-08-12 21:12:37,055 : INFO : PROGRESS: at sentence #50000, processed 834870 words, keeping 58363 word types
2019-08-12 21:12:40,619 : INFO : PROGRESS: at sentence #60000, processed 1002902 words, keeping 66765 word types
2019-08-12 21:12:44,200 : INFO : PROGRESS: at sentence #70000, processed 1175301 words, keeping 74771 word types
2019-08-12 21:12:47,761 : INFO : PROGRESS: a

2019-08-12 21:16:49,341 : INFO : PROGRESS: at sentence #710000, processed 13246147 words, keeping 629204 word types
2019-08-12 21:16:53,260 : INFO : PROGRESS: at sentence #720000, processed 13458288 words, keeping 638470 word types
2019-08-12 21:16:57,399 : INFO : PROGRESS: at sentence #730000, processed 13667846 words, keeping 649040 word types
2019-08-12 21:17:01,718 : INFO : PROGRESS: at sentence #740000, processed 13894371 words, keeping 658406 word types
2019-08-12 21:17:05,766 : INFO : PROGRESS: at sentence #750000, processed 14121254 words, keeping 668679 word types
2019-08-12 21:17:11,993 : INFO : PROGRESS: at sentence #760000, processed 14545179 words, keeping 679359 word types
2019-08-12 21:17:16,858 : INFO : PROGRESS: at sentence #770000, processed 14827121 words, keeping 690887 word types
2019-08-12 21:17:21,391 : INFO : PROGRESS: at sentence #780000, processed 15066747 words, keeping 702938 word types
2019-08-12 21:17:25,459 : INFO : PROGRESS: at sentence #790000, processe

2019-08-12 21:22:14,239 : INFO : PROGRESS: at sentence #1410000, processed 30691580 words, keeping 1374915 word types
2019-08-12 21:22:18,900 : INFO : PROGRESS: at sentence #1420000, processed 30935525 words, keeping 1384551 word types
2019-08-12 21:22:23,694 : INFO : PROGRESS: at sentence #1430000, processed 31175862 words, keeping 1394634 word types
2019-08-12 21:22:28,387 : INFO : PROGRESS: at sentence #1440000, processed 31420207 words, keeping 1404643 word types
2019-08-12 21:22:33,243 : INFO : PROGRESS: at sentence #1450000, processed 31682249 words, keeping 1413429 word types
2019-08-12 21:22:37,868 : INFO : PROGRESS: at sentence #1460000, processed 31911865 words, keeping 1422612 word types
2019-08-12 21:22:42,820 : INFO : PROGRESS: at sentence #1470000, processed 32188993 words, keeping 1433632 word types
2019-08-12 21:22:47,481 : INFO : PROGRESS: at sentence #1480000, processed 32427528 words, keeping 1443400 word types
2019-08-12 21:22:52,105 : INFO : PROGRESS: at sentence #

2019-08-12 21:28:16,558 : INFO : PROGRESS: at sentence #2110000, processed 51512590 words, keeping 2043706 word types
2019-08-12 21:28:24,188 : INFO : PROGRESS: at sentence #2120000, processed 51970661 words, keeping 2053165 word types
2019-08-12 21:28:31,337 : INFO : PROGRESS: at sentence #2130000, processed 52367134 words, keeping 2061262 word types
2019-08-12 21:28:35,665 : INFO : PROGRESS: at sentence #2140000, processed 52615955 words, keeping 2070634 word types
2019-08-12 21:28:40,054 : INFO : PROGRESS: at sentence #2150000, processed 52864506 words, keeping 2079900 word types
2019-08-12 21:28:44,493 : INFO : PROGRESS: at sentence #2160000, processed 53107623 words, keeping 2089533 word types
2019-08-12 21:28:48,683 : INFO : PROGRESS: at sentence #2170000, processed 53351207 words, keeping 2097912 word types
2019-08-12 21:28:52,904 : INFO : PROGRESS: at sentence #2180000, processed 53583193 words, keeping 2106172 word types
2019-08-12 21:28:57,428 : INFO : PROGRESS: at sentence #

2019-08-12 21:33:41,183 : INFO : PROGRESS: at sentence #2810000, processed 69024888 words, keeping 2592652 word types
2019-08-12 21:33:46,001 : INFO : PROGRESS: at sentence #2820000, processed 69252572 words, keeping 2600175 word types
2019-08-12 21:33:50,655 : INFO : PROGRESS: at sentence #2830000, processed 69506727 words, keeping 2606581 word types
2019-08-12 21:33:55,347 : INFO : PROGRESS: at sentence #2840000, processed 69738673 words, keeping 2613946 word types
2019-08-12 21:33:59,982 : INFO : PROGRESS: at sentence #2850000, processed 69982133 words, keeping 2621655 word types
2019-08-12 21:34:04,445 : INFO : PROGRESS: at sentence #2860000, processed 70196733 words, keeping 2627423 word types
2019-08-12 21:34:08,994 : INFO : PROGRESS: at sentence #2870000, processed 70435370 words, keeping 2634857 word types
2019-08-12 21:34:13,891 : INFO : PROGRESS: at sentence #2880000, processed 70746597 words, keeping 2641409 word types
2019-08-12 21:34:18,410 : INFO : PROGRESS: at sentence #

...

2019-08-13 05:34:45,619 : INFO : EPOCH 10 - PROGRESS: at 87.03% examples, 54307 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:46,780 : INFO : EPOCH 10 - PROGRESS: at 87.08% examples, 54304 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:48,776 : INFO : EPOCH 10 - PROGRESS: at 87.11% examples, 54275 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:49,865 : INFO : EPOCH 10 - PROGRESS: at 87.14% examples, 54275 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:50,873 : INFO : EPOCH 10 - PROGRESS: at 87.16% examples, 54276 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:52,015 : INFO : EPOCH 10 - PROGRESS: at 87.19% examples, 54278 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:53,029 : INFO : EPOCH 10 - PROGRESS: at 87.22% examples, 54280 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:54,136 : INFO : EPOCH 10 - PROGRESS: at 87.25% examples, 54279 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:34:55,268 : INFO : EPOCH 10 - PROGRESS: at 87.29% examples, 54277 words/s, in_qsiz

2019-08-13 05:36:06,893 : INFO : EPOCH 10 - PROGRESS: at 89.93% examples, 54237 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:08,031 : INFO : EPOCH 10 - PROGRESS: at 89.98% examples, 54236 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:09,112 : INFO : EPOCH 10 - PROGRESS: at 90.02% examples, 54235 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:10,280 : INFO : EPOCH 10 - PROGRESS: at 90.07% examples, 54233 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:11,337 : INFO : EPOCH 10 - PROGRESS: at 90.11% examples, 54233 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:12,349 : INFO : EPOCH 10 - PROGRESS: at 90.15% examples, 54230 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:13,515 : INFO : EPOCH 10 - PROGRESS: at 90.19% examples, 54227 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:14,705 : INFO : EPOCH 10 - PROGRESS: at 90.24% examples, 54225 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:36:15,833 : INFO : EPOCH 10 - PROGRESS: at 90.29% examples, 54223 words/s, in_qsiz

2019-08-13 05:37:27,491 : INFO : EPOCH 10 - PROGRESS: at 93.00% examples, 54165 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:28,595 : INFO : EPOCH 10 - PROGRESS: at 93.04% examples, 54164 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:29,647 : INFO : EPOCH 10 - PROGRESS: at 93.08% examples, 54161 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:30,741 : INFO : EPOCH 10 - PROGRESS: at 93.12% examples, 54157 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:31,901 : INFO : EPOCH 10 - PROGRESS: at 93.17% examples, 54155 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:32,999 : INFO : EPOCH 10 - PROGRESS: at 93.21% examples, 54154 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:34,142 : INFO : EPOCH 10 - PROGRESS: at 93.26% examples, 54153 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:35,329 : INFO : EPOCH 10 - PROGRESS: at 93.31% examples, 54150 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:37:36,478 : INFO : EPOCH 10 - PROGRESS: at 93.36% examples, 54148 words/s, in_qsiz

2019-08-13 05:38:49,456 : INFO : EPOCH 10 - PROGRESS: at 96.30% examples, 54056 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:50,612 : INFO : EPOCH 10 - PROGRESS: at 96.34% examples, 54054 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:51,736 : INFO : EPOCH 10 - PROGRESS: at 96.39% examples, 54053 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:52,886 : INFO : EPOCH 10 - PROGRESS: at 96.44% examples, 54051 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:53,893 : INFO : EPOCH 10 - PROGRESS: at 96.48% examples, 54049 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:55,080 : INFO : EPOCH 10 - PROGRESS: at 96.53% examples, 54046 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:56,104 : INFO : EPOCH 10 - PROGRESS: at 96.57% examples, 54044 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:57,285 : INFO : EPOCH 10 - PROGRESS: at 96.62% examples, 54041 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:38:58,308 : INFO : EPOCH 10 - PROGRESS: at 96.66% examples, 54039 words/s, in_qsiz

2019-08-13 05:40:10,299 : INFO : EPOCH 10 - PROGRESS: at 99.38% examples, 54031 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:11,335 : INFO : EPOCH 10 - PROGRESS: at 99.42% examples, 54032 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:12,472 : INFO : EPOCH 10 - PROGRESS: at 99.47% examples, 54031 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:13,664 : INFO : EPOCH 10 - PROGRESS: at 99.52% examples, 54029 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:14,839 : INFO : EPOCH 10 - PROGRESS: at 99.57% examples, 54026 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:15,851 : INFO : EPOCH 10 - PROGRESS: at 99.61% examples, 54024 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:16,993 : INFO : EPOCH 10 - PROGRESS: at 99.66% examples, 54022 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:18,129 : INFO : EPOCH 10 - PROGRESS: at 99.71% examples, 54021 words/s, in_qsize 0, out_qsize 0
2019-08-13 05:40:19,280 : INFO : EPOCH 10 - PROGRESS: at 99.76% examples, 54019 words/s, in_qsiz

In [29]:
w1 = "BPAG2"

print("Most similar to {0}".format(w1), model.wv.most_similar(positive=w1))

2019-08-13 08:01:45,516 : INFO : precomputing L2-norms of word weight vectors


Most similar to BPAG2 [('BP230', 0.6009641289710999), ('envoplakin', 0.5974247455596924), ('BPAG1', 0.5960305333137512), ('periplakin', 0.5906866788864136), ('BP180', 0.5779253840446472), ('desmoglein-1', 0.5593770742416382), ('DSG1', 0.5547298192977905), ('plakophilin', 0.5491119623184204), ('Dsg', 0.5460371971130371), ('desmoglein-3', 0.5376543998718262)]


In [30]:
w1 = "gut"

print("Most similar to {0}".format(w1), model.wv.most_similar(positive=w1))

Most similar to gut [('intestinal', 0.7504981160163879), ('ileal', 0.6605613231658936), ('commensal', 0.6538746356964111), ('colonic', 0.6279146671295166), ('dysbiotic', 0.6191140413284302), ('microbiome', 0.6187289357185364), ('microbiota', 0.6180603504180908), ('intestine', 0.6101160049438477), ('dysbiosis', 0.5952630639076233), ('mucosal', 0.5931606292724609)]


In [32]:
model.wv.save_word2vec_format('/home/stirunag/OTAR_Dataset/model_ex.bin', binary=True)

2019-08-13 08:43:16,006 : INFO : storing 2037314x200 projection weights into /home/stirunag/OTAR_Dataset/model_ex.bin


In [34]:
model.save('/home/stirunag/OTAR_Dataset/model.bin')

2019-08-13 08:44:09,877 : INFO : saving Word2Vec object under /home/stirunag/OTAR_Dataset/model.bin, separately None
2019-08-13 08:44:09,878 : INFO : storing np array 'vectors' to /home/stirunag/OTAR_Dataset/model.bin.wv.vectors.npy
2019-08-13 08:44:10,495 : INFO : not storing attribute vectors_norm
2019-08-13 08:44:10,496 : INFO : storing np array 'syn1neg' to /home/stirunag/OTAR_Dataset/model.bin.trainables.syn1neg.npy
2019-08-13 08:44:11,179 : INFO : not storing attribute cum_table
2019-08-13 08:44:15,755 : INFO : saved /home/stirunag/OTAR_Dataset/model.bin


In [14]:
bad = 'Crohnâ€™s disease'
good = bad.encode('cp1252').decode('utf-8')
print(good)


Crohn’s disease


In [16]:
# bad = 'decreased ERÃŽÂ±:ERÃŽÂ²'
bad = 'tyrosineÃ¢Â�Â´Ã‚Â¹Ã¢Â�Âµ'
good = bad.encode('cp1252').decode('utf-8')
print(good)

UnicodeEncodeError: 'charmap' codec can't encode character '\ufffd' in position 11: character maps to <undefined>

In [28]:
bad_sentence = 'Western blot revealed the tyrosineÃ¢Â�Â´Ã‚Â¹Ã¢Â�Âµ phosphorylation of tyrosine kinase c-Src under hypoxia.'
import re

new_sentence_list = []
for each_word in bad_sentence.split(' '):
    try:
        new_sentence_list.append(re.sub(r'[^\x00-\x7f]',r' ',each_word.encode('cp1252').decode('utf-8')).strip())   # Each char is a Unicode codepoint.
    except:
        new_sentence_list.append(re.sub(r'[^\x00-\x7f]',r' ',each_word).strip()) 

print(' '.join(new_sentence_list))    

Western blot revealed the tyrosine phosphorylation of tyrosine kinase c-Src under hypoxia.
